In [1]:
import re

import numpy as np

from matplotlib import pyplot as plt
from IPython.display import display, Markdown
from pathlib import Path
from collections import defaultdict

In [2]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 400

def show():
    plt.savefig(f"{show.fig_counter}.pdf", dpi='figure', transparent=True, bbox_inches='tight')
    show.fig_counter += 1
    show.plt_show()

show.fig_counter = 0
show.plt_show = plt.show

plt.show = show

In [3]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

In [4]:
BIODIVERSITY_PATTERN = re.compile(r"The simulation resulted in a biodiversity of (\d+) unique species\.")
EXECUTION_PATTERN = re.compile(r"The simulation took:\n - initialisation: ([^\n]+)\n - execution: ([^\n]+)\n - cleanup: ([^\n]+)\n")

In [5]:
BIODIVERSITY_PATTERN_NECSIM = re.compile(r"Number of species: (\d+)")
EXECUTION_PATTERN_NECSIM = re.compile(r"Setup time was (\d+) minutes (\d+) seconds\nSimulation time was (\d+) hours (\d+) minutes (\d+) seconds\nFile output and species calculation time was (\d+) minutes (\d+) seconds")

In [6]:
TIME_PATTERN = re.compile(r"(\d+(?:\.\d+)?)([^\d]+)")
TIME_UNITS = {
    "ns": 0.000000001,
    "µs": 0.000001,
    "ms": 0.001,
    "s": 1.0,
}

def parse_time(time_str):
    match = TIME_PATTERN.match(time_str)
    
    if match is None:
        return None
    
    return float(match.group(1)) * TIME_UNITS[match.group(2)]

In [7]:
SAMPLE_FILE_PATTERN = re.compile(r"pbs\.([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)\.o\d+\.\d+")

In [8]:
SAMPLE_FILE_PATTERN_NECSIM = re.compile(r"pbs\.([+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?)\.e\d+\.\d+")

In [9]:
plt.title("Execution Times (log-log) for the different algorithms\nand an increasing number of individuals on a single core")
plt.xlabel("number of individuals, increasingly densely packed")
plt.ylabel("time [s]")
plt.xscale("log")
plt.yscale("log")

plt.axhline(y=48*60*60, color='black', linestyle=':')
plt.text(2e6, 48*60*60, '48h HPC System Timeout', va='center', ha='center', backgroundcolor='white')

for algorithm, label in [("Gillespie", "Gillespie")]:
    results = defaultdict(list)

    for path in Path(CC_PATTERN.sub('-', algorithm).lower()).iterdir():
        match = SAMPLE_FILE_PATTERN.match(path.name)

        if match is None:
            continue

        sample = float(match.group(1))

        with open(path) as file:
            stdout = file.read()

        match = BIODIVERSITY_PATTERN.search(stdout)
        if match is None:
            continue
        biodiversity = int(match.group(1))

        match = EXECUTION_PATTERN.search(stdout)
        if match is None:
            continue
        initialisation = parse_time(match.group(1))
        execution = parse_time(match.group(2))
        cleanup = parse_time(match.group(3))

        results[sample].append((biodiversity, initialisation, execution, cleanup))

    results = { sample: [(np.median(s), np.mean(s), np.std(s)) for s in zip(*result)] for sample, result in results.items() }

    samples, biodiversities, initialisations, executions, cleanups = zip(*sorted([sample] + rs for sample, rs in results.items()))

    plt.plot([4.2e9 * s for s in samples], np.array(executions)[:,0], label=label)
    plt.fill_between(
        [4.2e9 * s for s in samples],
        np.array(executions)[:,1] - np.array(executions)[:,2],
        np.array(executions)[:,1] + np.array(executions)[:,2],
        color=plt.gca().lines[-1].get_color(), alpha=0.1
    )
    
for algorithm, label in [("necsim", "necsim [Thompson]")]:
    results = defaultdict(list)

    for path in Path(CC_PATTERN.sub('-', algorithm).lower()).iterdir():
        match = SAMPLE_FILE_PATTERN_NECSIM.match(path.name)

        if match is None:
            continue

        sample = float(match.group(1))

        with open(path) as file:
            stdout = file.read()

        match = BIODIVERSITY_PATTERN_NECSIM.search(stdout)
        if match is None:
            continue
        biodiversity = int(match.group(1))

        match = EXECUTION_PATTERN_NECSIM.search(stdout)
        if match is None:
            continue
        initialisation = int(match.group(1)) * 60 + int(match.group(2))
        execution = int(match.group(3)) * 60*60 + int(match.group(4)) * 60 + int(match.group(5))
        cleanup = int(match.group(6)) * 60 + int(match.group(7))

        results[sample].append((biodiversity, initialisation, execution, cleanup))

    results = { sample: [(np.median(s), np.mean(s), np.std(s)) for s in zip(*result)] for sample, result in results.items() }

    samples, biodiversities, initialisations, executions, cleanups = zip(*sorted([sample] + rs for sample, rs in results.items()))

    plt.plot([4.2e9 * s for s in samples], np.array(executions)[:,0], label=label)
    plt.fill_between(
        [4.2e9 * s for s in samples],
        np.array(executions)[:,1] - np.array(executions)[:,2],
        np.array(executions)[:,1] + np.array(executions)[:,2],
        color=plt.gca().lines[-1].get_color(), alpha=0.1
    )

for algorithm, label in [
    ("Classical", "Classical"), ("Independent", "Independent (CPU)"), ("SkippingGillespie", "SkippingGillespie"), ("Cuda", "Independent (GPU)")
]:
    results = defaultdict(list)

    for path in Path(CC_PATTERN.sub('-', algorithm).lower()).iterdir():
        match = SAMPLE_FILE_PATTERN.match(path.name)

        if match is None:
            continue

        sample = float(match.group(1))

        with open(path) as file:
            stdout = file.read()

        match = BIODIVERSITY_PATTERN.search(stdout)
        if match is None:
            continue
        biodiversity = int(match.group(1))

        match = EXECUTION_PATTERN.search(stdout)
        if match is None:
            continue
        initialisation = parse_time(match.group(1))
        execution = parse_time(match.group(2))
        cleanup = parse_time(match.group(3))

        results[sample].append((biodiversity, initialisation, execution, cleanup))

    results = { sample: [(np.median(s), np.mean(s), np.std(s)) for s in zip(*result)] for sample, result in results.items() }

    samples, biodiversities, initialisations, executions, cleanups = zip(*sorted([sample] + rs for sample, rs in results.items()))

    plt.plot([4.2e9 * s for s in samples], np.array(executions)[:,0], label=label)
    plt.fill_between(
        [4.2e9 * s for s in samples],
        np.array(executions)[:,1] - np.array(executions)[:,2],
        np.array(executions)[:,1] + np.array(executions)[:,2],
        color=plt.gca().lines[-1].get_color(), alpha=0.1
    )
    
    if algorithm == "Cuda":
        plt.text(4.2e9 * samples[-1] * 1.075, executions[-1][1] * 1.5, 'GPU machine\nRAM Limit', va='center', ha='center', color=plt.gca().lines[-1].get_color())
    
plt.legend()
plt.show()